<h1>Automated Gating with Immunova</h1>

This is the first of three experiments comparing the automated gating in immunova to expert manual gating. For this experiment I will be using pre-gated data from peritoneal dialysis patients.

<h3>Create project, experiments, and load fcs files</h3>

In [1]:
from immunova.data.project import Project
from immunova.data.mongo_setup import global_init
from tqdm import tqdm_notebook
global_init()

In [3]:
pd_project = Project(project_id='Peritonitis', owner='burtonrossj')
pd_project.save()

<Project: Project object>

In [4]:
pd_project.id.__str__()

'5d80d6cabcc1c27843c00893'

I will create four experiments in total:
* PBMC_T: T cell panel for PBMC samples
* PBMC_M: Myeloid cell panel for PBMC samples
* PDMC_T: T cell panel for peritoneal fluid samples
* PDMC_M: Myeloid cell panel for peritoneal fluid samples

For each of these experiments I need to associate a flow cytometry panel. A Panel object defines the channel(fluorochrome)/marker(antibody) mappings for all associated flow data. This allows for standardisation of the flow cytometry meta-data at the point of entry.

In [5]:
pd_project.add_experiment(experiment_id='PBMC_Tcells')

[]